In [53]:
#score: 0.91088
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, Bidirectional
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import itertools
import pandas as pd

In [49]:
# import mnist data and visualize first image
X_train = np.load("/kaggle/input/gundata/train_data.npy")
X_test = np.load("/kaggle/input/gundata/test_data.npy")
y_train = pd.read_csv('/kaggle/input/gundata/train_labels.csv').loc[:,'Label'].values

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train = np.reshape(X_train, (-1, 441,100))
X_test = np.reshape(X_test, (X_test.shape[0], 441,100))
#y_train = y_train.reshape(1, y_train.shape[0], 1)

In [50]:
print(y_train.shape)
print(X_train.shape)

(3999,)
(3999, 441, 100)


In [51]:
"""
print('Build LSTM RNN model ...')
model = Sequential()
model.add(LSTM(units=20, dropout=0.05, recurrent_dropout=0.20, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.20, return_sequences=False))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])

model.summary()
"""
# define LSTM
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2]), merge_mode='concat'))  
#model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.20, return_sequences=False))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_8 (Bidirection (None, 441, 128)          84480     
_________________________________________________________________
lstm_36 (LSTM)               (None, 32)                20608     
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 33        
Total params: 105,121
Trainable params: 105,121
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = 100, epochs = 50)

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = model.predict(X_test)
#np.savetxt("y_pred_ann.csv", y_pred, delimiter=",")
  
df2 = pd.DataFrame(y_pred, index = range(y_pred.shape[0]))
df2.reset_index(level=0, inplace=True)
df2.columns=['Id', 'Label']
df2.to_csv("y_pred_bidirectionalLSTM_64_32.csv", index=False)


Epoch 1/50
3999/3999 [==============================] - 83s 21ms/step - loss: 0.4162 - acc: 0.8507
Epoch 2/50
3999/3999 [==============================] - 77s 19ms/step - loss: 0.2671 - acc: 0.8780
Epoch 3/50
3999/3999 [==============================] - 77s 19ms/step - loss: 0.2385 - acc: 0.8765
Epoch 4/50
3999/3999 [==============================] - 77s 19ms/step - loss: 0.2417 - acc: 0.8770
Epoch 5/50
3999/3999 [==============================] - 77s 19ms/step - loss: 0.2338 - acc: 0.8772
Epoch 6/50
3999/3999 [==============================] - 76s 19ms/step - loss: 0.2277 - acc: 0.8792
Epoch 7/50
3999/3999 [==============================] - 77s 19ms/step - loss: 0.2231 - acc: 0.8787
Epoch 8/50
3999/3999 [==============================] - 77s 19ms/step - loss: 0.2204 - acc: 0.8782
Epoch 9/50
3999/3999 [==============================] - 77s 19ms/step - loss: 0.2179 - acc: 0.8785
Epoch 10/50
3999/3999 [==============================] - 76s 19ms/step - loss: 0.2161 - acc: 0.8805
Epoch 11/